# --- Day 16: Proboscidea Volcanium ---



In [180]:
def read_input(filename):
    flow_rates = dict()
    graph = dict()
    with open(filename) as infile:
        for line in infile:
            line = line.strip().replace("valves", "valve")
            line = line.strip().replace("tunnels lead", "tunnel leads")
            valve = line.split(" has flow rate=")[0].replace("Valve ", "")
            flow_rate, connections = line.split(" has flow rate=")[1].split(
                "; tunnel leads to valve "
            )
            flow_rates[valve] = int(flow_rate)
            graph[valve] = connections.split(", ")

    return flow_rates, graph


f, g = read_input("inputs/day16-example.txt")


In [181]:
from queue import Queue
import numpy as np


def get_valve_distances(G, root):
    # Breadth-first search
    # https://en.wikipedia.org/wiki/Breadth-first_search
    #
    # Input: A graph G and a starting vertex root of G
    # Output: Goal state. The parent links trace the shortest path back to root[8]

    # let Q be a queue
    Q = Queue()

    # Dict to store distances from root
    dist = {k: np.inf for k in G.keys()}

    # label root as explored
    explored = set()
    explored.update({0})

    # Q.enqueue(root)
    Q.put(root)

    while not Q.empty():
        # v := Q.dequeue()
        v = Q.get()

        # if v is the goal then return v
        if v == root:
            dist[v] = 0

        # for all edges from v to w in G.adjacentEdges(v)
        for w in G[v]:
            if w not in explored:
                if dist[w] > dist[v] + 1:
                    dist[w] = dist[v] + 1
                Q.put(w)

                # label w as explored
                explored.update({w})

    return dist


def get_useful_valve_distances(graph, flow_rates):
    # Distances between valves with non-zero flow rates
    dist = dict()
    for valve in graph.keys():
        if flow_rates[valve] > 0 or valve == "AA":
            dist[valve] = {
                k: v
                for k, v in get_valve_distances(graph, valve).items()
                if (f[k] > 0) and (k != valve)
            }

    return dist


def get_all_valve_distances(g):
    return {k: get_valve_distances(g, k) for k in g.keys()}


def get_path_time(path, distances):

    # Time to move between valves and open each valve
    return (
        sum([distances[path[i]][path[i + 1]] for i in range(0, len(path) - 1)])
        + len(path)
        - 1
    )


def calculate_path_value(path, distances, flow_rates, total_time=30):
    # Total pressure relief from a path
    elapsed_time = 0
    total_pressure = 0
    for i in range(1, len(path)):
        elapsed_time += distances[path[i - 1]][path[i]] + 1
        total_pressure += f[path[i]] * (total_time - elapsed_time)
    return total_pressure


# d = get_useful_valve_distances(g, f)
# calculate_path_value(["AA", "DD", "BB", "JJ", "HH", "EE", "CC"], d, f)


In [182]:
def run_part1(g, f):
    d = get_useful_valve_distances(g, f)
    i = 0
    max_time = 30
    paths = [["AA"]]

    while i < len(d):
        print(f"\nIteration: {i}")
        print(f"Paths: {len(paths)}")
        new_paths = list()
        while paths:
            path = paths.pop()
            new_branches = list()

            # Check if each path can be extended
            for v in d[path[-1]].keys():
                if v not in path and get_path_time(path + [v], d) < max_time:
                    new_branches.append(path + [v])

            # If we cannot extend this branch, keep original path
            if len(new_branches) == 0:
                new_branches = [path]

            new_paths += new_branches
        paths = new_paths
        i += 1

    return max([calculate_path_value(path, d, f) for path in paths])


f, g = read_input("inputs/day16.txt")
run_part1(g, f)



Iteration: 0
Paths: 1

Iteration: 1
Paths: 15

Iteration: 2
Paths: 210

Iteration: 3
Paths: 2627

Iteration: 4
Paths: 20443

Iteration: 5
Paths: 72978

Iteration: 6
Paths: 113249

Iteration: 7
Paths: 117487

Iteration: 8
Paths: 117503

Iteration: 9
Paths: 117503

Iteration: 10
Paths: 117503

Iteration: 11
Paths: 117503

Iteration: 12
Paths: 117503

Iteration: 13
Paths: 117503

Iteration: 14
Paths: 117503

Iteration: 15
Paths: 117503


1659